In [1]:
from __future__ import absolute_import
from __future__ import print_function
import os
import glob
import random
import numpy as np
from keras import optimizers
from keras.layers import LSTM
from keras.models import Sequential, Model
from keras.applications.vgg16 import VGG16
from keras.layers.wrappers import TimeDistributed
from keras.applications.mobilenet import MobileNet
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras.layers import Input, InputLayer
from keras.layers.core import Activation, Flatten, Reshape
from keras.layers.convolutional import Convolution2D, MaxPooling2D, UpSampling2D
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.applications import imagenet_utils
import h5py

Using TensorFlow backend.


In [3]:
videoFiles = glob.glob('../deep-dataset/numpys/*.npy')
mosFiles = [i for i in videoFiles if 'mos' in i]
videoFiles = [i for i in videoFiles if 'mos' not in i]

In [4]:
def myGenerator():
    while True:
        index_list = random.sample(range(1, 800), 2)
        alldata_x = []
        alldata_y = []
        for i in index_list:
            #print(i, index_list)
            f = videoFiles[i]
            s = f[:-4]+'_mos.npy'
            a = np.load(f)
            b = np.load(s)
            alldata_x.append(a)
            alldata_y.append(b)
        alldata_x = np.array(alldata_x)
        alldata_y = np.array(alldata_y)
        #print(alldata_x.shape, alldata_y.shape)
        yield alldata_x, alldata_y
# x = myGenerator()
# xtrain, ytrain = next(x)
# print('xtrain shape:',xtrain.shape)
# print('ytrain shape:',ytrain.shape)

In [4]:
print(len(videoFiles))
for i in videoFiles:
    print(np.load(i).shape)
    break

803
(200, 68, 120, 3)


In [2]:
height = 68
width = 120
input_shape=(200, height, width, 3)

In [5]:
def mySegNet(input_shape):
    base_model  = MobileNet(input_shape=(224,224,3), include_top=False)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    cnn_model = Model(inputs=base_model.input, outputs=x)
    
    model = Sequential();
    model.add(TimeDistributed(cnn_model, input_shape=input_shape))
    model.add(TimeDistributed(Flatten()))
    
    model.add(LSTM(512, return_sequences=False))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(5, activation='softmax'))
    model.compile(optimizer = optimizers.Adam(lr=0.00001), loss='categorical_crossentropy')
    print(model.summary())
    return model 
#mySegNet(input_shape)

In [6]:
m = mySegNet(input_shape)
m.layers.pop()
m.outputs = [m.layers[-1].output]
m.layers[-1].outbound_nodes = []
print(m.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_5 (TimeDist (None, 200, 1024)         3228864   
_________________________________________________________________
time_distributed_6 (TimeDist (None, 200, 1024)         0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 512)               3147776   
_________________________________________________________________
dense_5 (Dense)              (None, 100)               51300     
_________________________________________________________________
dense_6 (Dense)              (None, 5)                 505       
Total params: 6,428,445
Trainable params: 6,406,557
Non-trainable params: 21,888
_________________________________________________________________
None
_________________________________________________________________
Layer (type)                 Output Shape              P

In [ ]:
model = mySegNet(input_shape)
model.fit_generator(generator=myGenerator(),
                    use_multiprocessing=True,
                   steps_per_epoch=20, epochs=50)
model.fit_generator(generator=myGenerator(),
                    use_multiprocessing=True,
                   steps_per_epoch=20, epochs=50)
model.fit_generator(generator=myGenerator(),
                    use_multiprocessing=True,
                   steps_per_epoch=20, epochs=50)
# model.save('model1.h5')
# model.save_weights('model_weights1.h5')

In [28]:
x = myGenerator()
xtest, ytest = next(x)
pred = model.predict(xtest)
print(ytest, pred)

[[0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]] [[0.02274131 0.00705287 0.01550156 0.00471422 0.9499901 ]
 [0.00832896 0.00962861 0.0167392  0.00557504 0.9597282 ]]
